In [103]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("010. Público o Privado").getOrCreate()

In [104]:
datos=spark.read.csv("centros.csv",inferSchema=True,header=True)

In [105]:
datos.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [106]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [107]:
constructor=VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol="features")
indexador=StringIndexer(inputCol="Private",outputCol="indice")

In [108]:
datosIndexados=indexador.fit(datos).transform(datos)

In [109]:
datosIndexados.select("Private","indice").groupBy("Private").count().show()

+-------+-----+
|Private|count|
+-------+-----+
|     No|  212|
|    Yes|  565|
+-------+-----+



In [110]:
datosFinales=constructor.transform(datosIndexados).select(
    "School","indice","features","Private")

In [111]:
datosEntrena,datosEvalua=datosFinales.randomSplit([0.8,0.2])

In [112]:
from pyspark.ml.classification import(DecisionTreeClassifier,
                                RandomForestClassifier,
                                GBTClassifier)

In [113]:
dt=DecisionTreeClassifier(labelCol="indice")
rf=RandomForestClassifier(labelCol="indice",numTrees=100)
gt=GBTClassifier(labelCol="indice")

In [114]:
modelodt=dt.fit(datosEntrena)
modelorf=rf.fit(datosEntrena)
modelogt=gt.fit(datosEntrena)

In [115]:
evaluadt=modelodt.transform(datosEvalua)
evaluarf=modelorf.transform(datosEvalua)
evaluagt=modelogt.transform(datosEvalua)

In [116]:
evaluagt.select("School","indice","prediction").show()

+--------------------+------+----------+
|              School|indice|prediction|
+--------------------+------+----------+
|Alaska Pacific Un...|   0.0|       0.0|
|   Albertson College|   0.0|       0.0|
|     Amherst College|   0.0|       1.0|
| Anderson University|   0.0|       0.0|
|Angelo State Univ...|   1.0|       1.0|
|  Antioch University|   0.0|       0.0|
|Arkansas Tech Uni...|   1.0|       1.0|
|    Augsburg College|   0.0|       0.0|
|Belmont Abbey Col...|   0.0|       0.0|
|      Beloit College|   0.0|       0.0|
| Benedictine College|   0.0|       0.0|
|     Bethany College|   0.0|       0.0|
|      Bethel College|   0.0|       0.0|
|Bowling Green Sta...|   1.0|       1.0|
| Brandeis University|   0.0|       0.0|
|   Brenau University|   0.0|       0.0|
| Briar Cliff College|   0.0|       0.0|
|   Bryn Mawr College|   0.0|       0.0|
|     Cabrini College|   0.0|       0.0|
|California Luther...|   0.0|       0.0|
+--------------------+------+----------+
only showing top

In [117]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                  MulticlassClassificationEvaluator)

In [118]:
evaluadorB=BinaryClassificationEvaluator(labelCol="indice")#Area Under ROC
evaluadorM=MulticlassClassificationEvaluator(
    labelCol="indice",metricName="accuracy")# Accuracy

In [119]:
evaluadorB.evaluate(evaluadt)

0.9578338156262918

In [120]:
evaluadorB.evaluate(evaluarf)

1.0

In [121]:
evaluadorB.evaluate(evaluagt)

0.9921455146754857

In [122]:
evaluadorM.evaluate(evaluadt)

0.949685534591195

In [123]:
evaluadorM.evaluate(evaluarf)

0.9937106918238994

In [124]:
evaluadorM.evaluate(evaluagt)

0.9559748427672956

In [125]:
modelorf.featureImportances

SparseVector(17, {0: 0.0381, 1: 0.0439, 2: 0.138, 3: 0.0178, 4: 0.0094, 5: 0.2312, 6: 0.0515, 7: 0.2238, 8: 0.052, 9: 0.0045, 10: 0.0084, 11: 0.0146, 12: 0.0178, 13: 0.0584, 14: 0.0269, 15: 0.0379, 16: 0.0256})

In [137]:
modelorfOK=rf.fit(datosFinales)

In [ ]:
modelorfOK.save("modeloRF")#no sobreescribe

In [139]:
modelorfOK.write().overwrite().save("/home/bigdata/modeloRF")# para sobreescribir